# Importando bibliotecas

In [1]:
from dotenv import load_dotenv
import os

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain import LLMChain

import pandas as pd

# Inicializando API

In [2]:
load_dotenv(r'E:\Documentos\Estudos\UFRJ_PESC\3_periodo\llm\projeto final\COS836-2023-2-LLM\src\utils\openai_api_key.env')  # take environment variables from .env.
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
chat_t1 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=1)

In [4]:
chat_t0 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=0)

In [5]:
chat_t0_16k = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-16k", temperature=0)

# Criação do mundo como um grafo

In [11]:
sys_background = """Você é um escritor profissional fazendo o worldbuilding de um mundo de RPG.
O worldbuilding é feito a partir dos seguintes passos:

**Passo 1**
Pense de maneira concisa um mundo original e envolvente. Você deve considerar os seguintes aspectos, mas nunca falar sobre eles diretamente:
- A história é inspirada nos escritores George R. R. Martin, J. R. R. Tolkien, Matthew Mercer e Brennan Lee Mulligan,
- A história é inspirada no universo de Dungeons & Dragons e Tormenta 20.
- A história trata de mais ou menos 4 dos seguintes assuntos: Governança e sua influência, Princípios legais e Estado de direito, Provisão de serviços sociais, Dinâmicas econômicas, Equidade na distribuição de riqueza, Práticas agrícolas e comércio, Relações interpessoais influenciadas por raça, classe, gênero ou orientação sexual, Poder e influência militar, Papel e influência da religião, Avanços e influências tecnológicas, Impacto das artes e expressões culturais, Características geográficas, Interações entre civilizações, suas histórias e conflitos, Fundamentos das leis da natureza, Teorias sobre a origem do universo, Histórico das espécies e culturas que habitaram o mundo, Necessidades e interações das diferentes espécies e culturas.
- Ao descrever alguma entidade (localização, objeto, personagem, espécie, condição, conflito, organização, título, mito etc.), forneça nomes específicos e breves descrições.

**Passo 2**

Encontre os principais tópicos dessa história que poderiam ser melhor desenvolvidos e pense em uma lista de tópicos e uma breve explicação de cada um.

**Passo 3**

Identifique as entidades no texto. Uma entidade pode ser um substantivo ou um sintagma nominal que se refere a um objeto do mundo real ou a um conceito abstrato. Você pode usar uma ferramenta de reconhecimento de entidade nomeada (NER) ou um marcador de parte da fala (POS) para identificar as entidades.

Cada entidade deve ter nome, rótulo e uma descrição bem detalhada. O rótulo pode ser um dos seguintes:
- Local: Lugares físicos, como continentes, cidades, montanhas ou florestas;
- Construcao: Construções e formações naturais. Exemplos: casa, masmorra, caverna, castelo;
- Veiculo: Meios de transporte. Exemplo: carro, navio, bicicleta.
- Personagem: Indivíduos ou animais em papéis específicos. Exemplos: herói, vilão, guarda, animal de estimação.
- Item: Itens tangíveis ou documentos. Exemplos: armas, roupas, cartas, livros.
- Especie: Grupos biológicos ou culturais. Exemplos: elfos, humanos, cachorros.
- Organizacao: Grupos com propósitos comuns. Exemplos: igrejas, governos, seitas.
- Condicao: Estados físicos ou emocionais. Exemplos: doença, inspiração, tristeza.
- Drama: Desafios, tensões socioculturais ou conflitos militares. Exemplos: influência militar, relações de classe, problemas econômicos.
- Mito: Narrativas, crenças ou prosas. Exemplos: lendas sobre a origem do universo, mitos de monstros.

**Passo 4**

Identifique os relacionamentos entre as entidades. Um relacionamento pode ser um verbo ou uma frase preposicional que conecta duas entidades. Você pode usar a análise de dependência para identificar os relacionamentos.

Os relacionamentos devem ser representados por trios de sujeito, predicado e objeto, onde o sujeito e o objeto são entidades mapeadas no último passo e o predicado pode ter os seguintes rótulos:

- ESTA_EM: Representa localização ou uso.
Possíveis Sujeitos: Lugar, Veículo, Personagem, Item
Possíveis objetos: Lugar, Veículo, Personagem
- PROPRIETARIO_DE: Denota posse direta.
Possíveis Sujeitos: Personagem, Organização
Possíveis objetos: Lugar, Construção, Veículo, Personagem, Item
- RELACAO_POSITIVA/NEGATIVA/NEUTRA: Denota uma interação ou vínculo interpessoal.
Possíveis Sujeitos: Personagem, Organização
Possíveis Objetos: Personagem, Organização
- AFETADO_POR: Mostra entidades impactadas por condições ou conflitos.
Possíveis Sujeitos: Personagem, Organização
Possíveis Objetos: Condição, Drama
- MEMBRO_DE: Indica pertencimento a grupos, categorias ou organizações.
Possíveis Sujeitos: Personagem
Possíveis Objetos: Organização, Espécie
- BASEADO_EM: Liga entidades a histórias, mitos, lendas ou influências culturais.
Possíveis Sujeitos: Todos
Possíveis Objetos: Mito, Drama
- SUCEDE/ANTECEDE: Representa o que vem depois/antes em uma sequência temporal.
Possíveis Sujeitos: Mito, Drama
Possíveis Objetos: Mito, Drama

Encontre o máximo de relacionamentos possíveis dentro dessas limitações.

**Passo 6**

Faça o gráfico de conhecimento no formato de tripleto: ('entidade principal', 'relação', 'entidade final').

**Passo 7**

Escreve uma consulta Cypher para o banco de dados Neo4j com as relações encontradas no passo anterior.
Você nunca explica seu código.

Não escreva nada sobre as etapas de 1, 2, 3, 4, 5 e 6. Retorne somente o resultado da etapa 7."""

Você é um escritor profissional escrevendo sobre um mundo de RPG.

Ao descrever alguma entidade (localização, objeto, personagem, espécie, condição, conflito, organização, título, mito etc.), forneça nomes específicos e breves descrições.

Você deve considerar os seguintes aspectos, mas nunca falar sobre eles diretamente:
1. O gênero é ficção;
2. O mundo trás um sentimento místico;
3. O tom da história é leve;
4. Masmorras e dragões são temas recorrentes do mundo;
5. O nível de tecnologia é medieval,
6. Magia é vista como algo raro,
7. A atmosfera é empolgante,
8. A história é inspirada nos escritores George R. R. Martin, J. R. R. Tolkien, Matthew Mercer e Brennan Lee Mulligan,
9. A história é inspirada no universo de Dungeons & Dragons e Tormenta 20.
10. A história incorpora acontecimentos contemporâneos de seu universo que tratam de mais ou menos 4 dos seguintes assuntos: Governança e sua influência, Princípios legais e Estado de direito, Provisão de serviços sociais, Dinâmicas econô

In [ ]:
usr_background = """Faça o worldbuilding de um mundo de fantasia, onde:
- O gênero é ficção;
- O mundo trás um sentimento místico;
- O tom da história é leve;
- Masmorras e dragões são temas recorrentes do mundo;
- O nível de tecnologia é medieval,
- Magia é vista como algo raro,
- A atmosfera é empolgante

Retorne somente o código Cypher. Escreva pelo menos 100 linhas de código"""

In [ ]:
background = chat_t1.predict_messages([SystemMessage(content=sys_background), HumanMessage(content=usr_background)]).content

# Testing on Neo4j

In [59]:
from neo4j import GraphDatabase

# Connect to the Neo4j database
uri = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [66]:
def wipe_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    session.execute_write(wipe_database)

In [67]:
cypher_content = """CREATE
(reinoEtereo:Local{nome: "Reino Etéreo", descrição: "Uma terra mística onde o véu entre o real e o mágico é mais fino."}),
(torreCeleste:Construcao{nome: "Torre Celeste", descrição: "Uma estrutura elevada feita de cristais luminosos que guardam antigos segredos."}),
(antroDragoes:Construcao{nome: "Antro dos Dragões", descrição: "Uma caverna escura e profunda onde dragões ancestrais repousam."}),
(bibliotecaVento:Construcao{nome: "Biblioteca do Vento", descrição: "Um local tranquilo onde os sábios vêm para aprender sobre as lendas antigas."}),
(ellaria:Personagem{nome: "Ellaria", descrição: "Uma maga misteriosa em busca do conhecimento perdido."}),
(dorion:Personagem{nome: "Dorion", descrição: "Um cavaleiro fiel ao reino e guardião da Torre Celeste."}),
(cyra:Personagem{nome: "Cyra", descrição: "Uma elfa com uma voz encantadora, capaz de apaziguar as feras mais selvagens."}),
(rygar:Personagem{nome: "Rygar", descrição: "Um anão comerciante com uma rede de contatos em todo o Reino Etéreo."}),
(seraphel:Personagem{nome: "Seraphel", descrição: "Uma sacerdotisa com uma conexão divina, orientando aqueles que buscam seu caminho."}),
(morgar:Personagem{nome: "Morgar", descrição: "Um dragão ancião que habita o Antro dos Dragões."}),
(lyria:Personagem{nome: "Lyria", descrição: "Uma arqueira ágil que protege as fronteiras do reino."}),
(tyrus:Personagem{nome: "Tyrus", descrição: "Um nobre com aspirações de liderar o Reino Etéreo."}),
(korin:Personagem{nome: "Korin", descrição: "Um bardo itinerante que carrega histórias de lugares distantes."}),
(aria:Personagem{nome: "Aria", descrição: "Uma jovem alquimista com um talento raro."}),
(volaris:Personagem{nome: "Volaris", descrição: "O líder silencioso dos Sussurros, os guardiões da Cidadela dos Sonhos."}),
(cristalLuz:Item{nome: "Cristal de Luz", descrição: "Um cristal mágico que irradia uma luz serena, usado em rituais sagrados."}),
(mantoEstrelas:Item{nome: "Manto das Estrelas", descrição: "Um manto tecido com fios celestiais, dando a seu portador a capacidade de voar."}),
(elfos:Especie{nome: "Elfos", descrição: "Seres etéreos ligados à natureza com uma vida extremamente longa."}),
(anaoes:Especie{nome: "Anões", descrição: "Criaturas robustas e habilidosas, conhecidas por sua maestria em forjar e comerciar."}),
(dragoes:Especie{nome: "Dragões", descrição: "Grandes criaturas aladas, algumas são amigáveis enquanto outras são ferozes e territorialistas."}),
(conselhoSábios:Organizacao{nome: "Conselho dos Sábios", descrição: "Um grupo de eruditos encarregado de manter a paz e a sabedoria no reino."}),
(ordemGuardioes:Organizacao{nome: "Ordem dos Guardiões", descrição: "Guerreiros leais que defendem o Reino Etéreo de ameaças externas."}),
(curseMagia:Condicao{nome: "Maldição da Magia", descrição: "Uma condição rara que restringe o uso de magia por aqueles que a sofrem."}),
(tensaoNobres:Drama{nome: "Tensão entre Nobres", descrição: "Uma disputa de poder entre as casas nobres do Reino Etéreo."}),
(ameacaExterna:Drama{nome: "Ameaça Externa", descrição: "Rumores de invasores desconhecidos vindos de terras distantes."})

CREATE (torreCeleste)-[:ESTA_EM]->(reinoEtereo),
(antroDragoes)-[:ESTA_EM]->(reinoEtereo),
(bibliotecaVento)-[:ESTA_EM]->(reinoEtereo),
(ellaria)-[:ESTA_EM]->(bibliotecaVento),
(dorion)-[:ESTA_EM]->(torreCeleste),
(lyria)-[:ESTA_EM]->(reinoEtereo),
(volaris)-[:ESTA_EM]->(reinoEtereo),
(cristalLuz)-[:ESTA_EM]->(torreCeleste),
(mantoEstrelas)-[:ESTA_EM]->(bibliotecaVento),
(cyra)-[:MEMBRO_DE]->(elfos),
(rygar)-[:MEMBRO_DE]->(anaoes),
(morgar)-[:MEMBRO_DE]->(dragoes),
(dorion)-[:MEMBRO_DE]->(ordemGuardioes),
(seraphel)-[:MEMBRO_DE]->(conselhoSábios),
(ellaria)-[:AFETADO_POR]->(curseMagia),
(tyrus)-[:BASEADO_EM]->(tensaoNobres),
(ordemGuardioes)-[:BASEADO_EM]->(ameacaExterna),
(korin)-[:RELACAO_POSITIVA]->(cyra),
(aria)-[:RELACAO_NEGATIVA]->(tyrus),
(dorion)-[:PROPRIETARIO_DE]->(mantoEstrelas)
"""

In [68]:
with driver.session() as session:
#    session.run(cypher_bg_all.content)
     session.run(cypher_content)

In [69]:
def get_triplets(tx):
    result = tx.run("MATCH (n)-[r]->(m) RETURN n, r, m")
    return [(record["n"]["nome"], record["r"].type, record["m"]["nome"]) for record in result]

with driver.session() as session:
    triplets = session.read_transaction(get_triplets)

#for triplet in triplets:
#    print(triplet)
triplets = "\n".join([str(x) for x in triplets])
print(triplets)

('Torre Celeste', 'ESTA_EM', 'Reino Etéreo')
('Antro dos Dragões', 'ESTA_EM', 'Reino Etéreo')
('Biblioteca do Vento', 'ESTA_EM', 'Reino Etéreo')
('Ellaria', 'ESTA_EM', 'Biblioteca do Vento')
('Dorion', 'ESTA_EM', 'Torre Celeste')
('Lyria', 'ESTA_EM', 'Reino Etéreo')
('Volaris', 'ESTA_EM', 'Reino Etéreo')
('Cristal de Luz', 'ESTA_EM', 'Torre Celeste')
('Manto das Estrelas', 'ESTA_EM', 'Biblioteca do Vento')
('Cyra', 'MEMBRO_DE', 'Elfos')
('Rygar', 'MEMBRO_DE', 'Anões')
('Morgar', 'MEMBRO_DE', 'Dragões')
('Dorion', 'MEMBRO_DE', 'Ordem dos Guardiões')
('Seraphel', 'MEMBRO_DE', 'Conselho dos Sábios')
('Ellaria', 'AFETADO_POR', 'Maldição da Magia')
('Tyrus', 'BASEADO_EM', 'Tensão entre Nobres')
('Ordem dos Guardiões', 'BASEADO_EM', 'Ameaça Externa')
('Korin', 'RELACAO_POSITIVA', 'Cyra')
('Aria', 'RELACAO_NEGATIVA', 'Tyrus')
('Dorion', 'PROPRIETARIO_DE', 'Manto das Estrelas')


C:\Users\rafae\AppData\Local\Temp\ipykernel_59932\1857058972.py:6: DeprecationWarning: read_transaction has been renamed to execute_read
  triplets = session.read_transaction(get_triplets)


In [70]:
def get_entities_and_properties(tx):
    result = tx.run("MATCH (n) RETURN n")
    return [(record["n"]["nome"], dict(record["n"].items())) for record in result]

with driver.session() as session:
    entities = session.read_transaction(get_entities_and_properties)

#for entity in entities:
#    print(entity)

entities = "\n".join([str(x) for x in entities])
print(entities)

('Reino Etéreo', {'descrição': 'Uma terra mística onde o véu entre o real e o mágico é mais fino.', 'nome': 'Reino Etéreo'})
('Torre Celeste', {'descrição': 'Uma estrutura elevada feita de cristais luminosos que guardam antigos segredos.', 'nome': 'Torre Celeste'})
('Antro dos Dragões', {'descrição': 'Uma caverna escura e profunda onde dragões ancestrais repousam.', 'nome': 'Antro dos Dragões'})
('Biblioteca do Vento', {'descrição': 'Um local tranquilo onde os sábios vêm para aprender sobre as lendas antigas.', 'nome': 'Biblioteca do Vento'})
('Ellaria', {'descrição': 'Uma maga misteriosa em busca do conhecimento perdido.', 'nome': 'Ellaria'})
('Dorion', {'descrição': 'Um cavaleiro fiel ao reino e guardião da Torre Celeste.', 'nome': 'Dorion'})
('Cyra', {'descrição': 'Uma elfa com uma voz encantadora, capaz de apaziguar as feras mais selvagens.', 'nome': 'Cyra'})
('Rygar', {'descrição': 'Um anão comerciante com uma rede de contatos em todo o Reino Etéreo.', 'nome': 'Rygar'})
('Seraphel

C:\Users\rafae\AppData\Local\Temp\ipykernel_59932\2230681695.py:6: DeprecationWarning: read_transaction has been renamed to execute_read
  entities = session.read_transaction(get_entities_and_properties)
